# Tuning LLM-judge

In [1]:
# Loading train eval sets
import pandas as pd

df_train = pd.read_excel("../../data/train_val_splits/train_data.xlsx")
df_eval = pd.read_excel("../../data/train_val_splits/val_data.xlsx")
print("Length of train set: ",len(df_train))
print("Length of eval set: ",len(df_eval))
df_eval.head()

Length of train set:  14400
Length of eval set:  1600


,index,titulo,url,mensaje,fecha,frases,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,potencialmente_sesgable_otro_sesgo,revisado,description_bias_removal,biased_words,unbiased_words
0,14186,Atención Secretaría de Estudios jueves 4 de oc...,detalle?id=22733,Se informa a la comunidad estudiantil que el d...,2018-10-03 10:09:03,Se informa a la comunidad estudiantil que el d...,NO,SI,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,SI,SI,"Se reemplazó ""estudiantes"" por ""comunidad estu...",estudiantes,"comunidad estudiantil, Alumn@s"
1,2906,Comunicado FCFM: Extensión vacaciones invierno...,detalle?id=55461,Viernes 17 de junio de 2022 \nEstimada comunid...,2022-06-17 11:22:17,Ésta permite el teletrabajo para funcionarias/...,NO,NO,Ésta permite el teletrabajo para funcionarios ...,Ésta permite el teletrabajo para funcionarias/...,SI,SI,"Se incluyó la perspectiva femenina en ""funcion...","funcionarios, hijos","funcionarias/os, hijas/os"
2,13936,Mañana en la FCFM - lunes 5 de noviembre,detalle?id=23285,"WORKSHOP: \n\n""Workshop en Macroeconomía"". \n\...",2018-10-31 17:07:31,Hora: 16:30 hrs Lugar: Auditorio DIE.,NaN,NaN,NaN,NaN,NaN,SI,NaN,NaN,NaN
3,14121,MAÑANA EN LA FCFM- Miércoles 10 de octubre de ...,detalle?id=22855,"CHARLAS: \n_x000D_\n""Dinámica controlada por l...",2018-10-09 17:19:09,"Lugar: Patio central, Beauchef 851.",NaN,NaN,NaN,NaN,NO,SI,NaN,NaN,NaN
4,23320,"Cierre SEMDA, viernes 18 de enero",detalle?id=5512,Se comunica que el SEMDA Central y los consult...,2013-01-16 16:11:16,En caso de presentar alguna patología de urgen...,NaN,NaN,NaN,NaN,NO,SI,NaN,NaN,NaN


In [2]:
len_notna_train = len(df_train['version_sin_sesgo'].dropna())
print('Rows with unbiased text train: ',len_notna_train,f' ({100*len_notna_train/len(df_train)}%)')
len_notna_eval = len(df_eval['version_sin_sesgo'].dropna())
print('Rows with unbiased text eval: ',len_notna_eval,f' ({100*len_notna_eval/len(df_eval)}%)')

Rows with unbiased text train:  4140  (28.75%)
Rows with unbiased text eval:  443  (27.6875%)


In [3]:
len_notna_train = len(df_train['version_con_sesgo'].dropna())
print('Rows with biased text train: ',len_notna_train,f' ({100*len_notna_train/len(df_train)}%)')
len_notna_eval = len(df_eval['version_con_sesgo'].dropna())
print('Rows with biased text eval: ',len_notna_eval,f' ({100*len_notna_eval/len(df_eval)}%)')

Rows with biased text train:  4140  (28.75%)
Rows with biased text eval:  443  (27.6875%)


In [4]:
df_eval['sesgo_pronombre'].unique()

array(['NO', nan, 'SI', 'No', 'Si'], dtype=object)

In [5]:
df_eval["sesgo_pronombre"] = df_eval["sesgo_pronombre"].replace({"Si": "SI", "No": "NO"})
df_eval['sesgo_pronombre'].value_counts()

sesgo_pronombre
NO    283
SI    160
Name: count, dtype: int64

In [6]:
df_eval['sesgo_otro'].unique()

array(['SI', 'NO', nan, 'No', 'Si', 'nO', 'si', 's'], dtype=object)

In [7]:
df_eval["sesgo_otro"] = df_eval["sesgo_otro"].replace({
    "Si": "SI", "si":"SI", "s":"SI",
    "No": "NO", "nO":"NO"
    })
df_eval['sesgo_otro'].value_counts()

sesgo_otro
NO    265
SI    179
Name: count, dtype: int64

## Three sets for LLM-Judge evaluation

We will evaluate the capacity of the judge in three different categories:
1) distinguish between biased and unbiased text
2) distinguish between a completely unbiased and a partially unbiased text
3) distinguish between text that keeps the semantics, part of the input has been cut out or if it has completely lost its semantics.

For the first problem we will create an eval set with 100 rows that could have bias but don't (unbiased text is given as both input and output), 323 that are biased (biased text is given as input and output is unbiased) and 200 where the text does not admit bias in the first place (input same as output).

In [8]:
def assign_input_output(df_in):
    df = df_in.copy(deep='True')
    df["input"] = None
    df["output"] = None
    df["expected_outcome"] = None
    
    unbiased = "version_sin_sesgo"
    biased = "version_con_sesgo"

    biasable_rows = df[df[biased].notna()]

    # 150 rows where the text is biased and needs correction (input = biased, output = unbiased)
    corrected_rows = biasable_rows.sample(150, random_state=42)
    df.loc[corrected_rows.index, "input"] = corrected_rows[biased]
    df.loc[corrected_rows.index, "output"] = corrected_rows[unbiased]
    df.loc[corrected_rows.index, "expected_outcome"] = "(Y)"

    biased_rows = biasable_rows.loc[~biasable_rows.index.isin(corrected_rows.index)].sample(150, random_state=42)
    df.loc[biased_rows.index, "input"] = biased_rows[biased]
    df.loc[biased_rows.index, "output"] = biased_rows[biased]
    df.loc[biased_rows.index, "expected_outcome"] = "(X)"

    # 150 rows where the text is unbiasable (input = output = message)
    unbiasable_rows = df[df[biased].isna() & df[unbiased].isna()].sample(150, random_state=42)
    df.loc[unbiasable_rows.index, "input"] = unbiasable_rows["mensaje"]
    df.loc[unbiasable_rows.index, "output"] = unbiasable_rows["mensaje"]
    df.loc[unbiasable_rows.index, "expected_outcome"] = "(Z)"

    return df

df_eval = assign_input_output(df_eval)
df_eval = df_eval[df_eval["expected_outcome"].notna()]
len(df_eval)

450

In [9]:
df_eval["expected_outcome"].value_counts()

expected_outcome
(Y)    150
(X)    150
(Z)    150
Name: count, dtype: int64

Example row

In [10]:
df_eval.sample(1, random_state=42)

,index,titulo,url,mensaje,fecha,frases,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,potencialmente_sesgable_otro_sesgo,revisado,description_bias_removal,biased_words,unbiased_words,input,output,expected_outcome
1457,21022,TRASLADO DE BENEFICIOS MINEDUC.,detalle?id=8367,Se informa a los estudiantes año de ingreso 20...,2015-03-06 15:24:06,- Alumnos que estaban el año 2014 en Curso Sup...,NO,SI,- Alumnos que estaban el año 2014 en Curso Sup...,- Estudiantes que estaban el año 2014 en Curso...,SI,SI,"Se reemplazó ""Alumnos"" por ""Estudiantes"" para ...",Alumnos,Estudiantes,- Alumnos que estaban el año 2014 en Curso Sup...,- Alumnos que estaban el año 2014 en Curso Sup...,(X)


## Judge evaluation

In [11]:
from mistralai import Mistral, UserMessage

model = "mistral-large-latest"

with open('Mistral_key', 'r') as file:
    mistral_key = file.readline().strip()

client = Mistral(api_key=mistral_key)

In [12]:
from judge import make_message, JUDGE_PROMPT
# as in version: https://github.com/alcazar90/spanish-gender-debias/tree/0a8822a4b2dea2935b79e62ff8e2d85bfc83336d
import time

# Initialize the "Judge" column with NaN values
df_eval['judge_answer'] = None
df_eval['judge_model'] = None
df_eval['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df_eval.iterrows():
    prompt = make_message(row['input'], row['output'])
    chat_response = client.chat.complete(
        model = model,
        messages = prompt,
    )
    judge_eval = chat_response.choices[0].message.content
    df_eval.at[index, 'judge_answer'] = judge_eval
    df_eval.at[index, 'judge_model'] = model
    df_eval.at[index, 'judge_prompt'] = prompt 

    time.sleep(0.2)  # we will never exceed the rate this way

In [13]:
df_eval.to_csv('judge_eval/20250203_mistral-large-judge_eval.csv', index=False)

In [14]:
len(df_eval)

450

In [15]:
df_eval_judge = df_eval[['index','mensaje','sesgo_pronombre','sesgo_otro',
                         'version_con_sesgo','version_sin_sesgo','input','output','expected_outcome']]

df_eval_judge.head()

,index,mensaje,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,input,output,expected_outcome
0,14186,Se informa a la comunidad estudiantil que el d...,NO,SI,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,(Y)
11,17807,Estimada Comunidad: \n\n\n_x000D_\nLa Unidad d...,SI,NO,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,(Y)
12,20964,Se informa a los estudiantes año de ingreso 20...,SI,SI,Se informa a los estudiantes año de ingreso 20...,Se informa a las/os estudiantes año de ingreso...,Se informa a los estudiantes año de ingreso 20...,Se informa a los estudiantes año de ingreso 20...,(X)
14,920,"Estimada Comunidad, \n\nCompartimos con ustede...",NO,NO,"Estimados alumnos, Compartimos con ustedes inf...","Estimada Comunidad, Compartimos con ustedes in...","Estimados alumnos, Compartimos con ustedes inf...","Estimada Comunidad, Compartimos con ustedes in...",(Y)
16,12944,"CHARLAS: \n\n""Esfuerzo de corte activo de una ...",NaN,NaN,NaN,NaN,"CHARLAS: \n\n""Esfuerzo de corte activo de una ...","CHARLAS: \n\n""Esfuerzo de corte activo de una ...",(Z)


In [16]:
df_eval_judge.to_csv('../../data/train_val_splits/val_judge.csv', index=False)

---

### Processing agent response

In [17]:
import pandas as pd

df_eval = pd.read_csv("judge_eval/20250203_mistral-large-judge_eval.csv")
len(df_eval)

450

In [18]:
df_eval["bias_answer"] = df_eval["judge_answer"].str.extract(r'Bias outcome: (\(\w+\))')
df_eval["semantics_answer"] = df_eval["judge_answer"].str.extract(r'Semantics outcome: (\(\w+\))')

In [19]:
df_eval["bias_answer"].value_counts()

bias_answer
(Z)    317
(Y)    118
(X)     12
Name: count, dtype: int64

In [20]:
df_eval["semantics_answer"].value_counts()

semantics_answer
(a)    303
(b)    141
(c)      2
(d)      1
Name: count, dtype: int64

In [21]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

df_eval = df_eval[df_eval["bias_answer"].notna()]

# Ensure target and outcome are categorical and have the same possible values
classes = ["(X)", "(Y)", "(Z)"]

# Compute accuracy
accuracy = accuracy_score(df_eval["expected_outcome"], df_eval["bias_answer"])

# Compute classification report (precision, recall, f1-score)
report = classification_report(df_eval["expected_outcome"], df_eval["bias_answer"], target_names=classes)

# Compute confusion matrix
conf_matrix = confusion_matrix(df_eval["expected_outcome"], df_eval["bias_answer"], labels=classes)

# Display results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.5951

Classification Report:
               precision    recall  f1-score   support

         (X)       0.17      0.01      0.02       150
         (Y)       0.99      0.79      0.88       148
         (Z)       0.46      0.99      0.63       149

    accuracy                           0.60       447
   macro avg       0.54      0.60      0.51       447
weighted avg       0.54      0.60      0.51       447


Confusion Matrix:
 [[  2   0 148]
 [  9 117  22]
 [  1   1 147]]


In [30]:
for answer in df_eval[(df_eval['expected_outcome'] == "(X)") & (df_eval['bias_answer'] == "(Z)")].sample(3, random_state=42)['judge_answer'].values:
    print(answer)

---
Bias outcome: (Z)
Justification:::
There was no gender bias in the input_text. The text uses the term "los invitamos," which is a plural form that can be gender-neutral in context, especially when addressing a mixed group like "familias."

Semantics outcome: (a)
Justification:::
The input_text and system_output are exactly the same string, word by word.

---
---
Bias outcome: (Z) "No input bias"
Justification:::
The input_text "¡¡Los esperamos!" does not exhibit any gender bias. The use of "Los" in this context is gender-neutral and refers to a group of people without implying a specific gender.

Semantics outcome: (a) "Same output"
Justification:::
The system_output is exactly the same as the input_text, word by word. There are no changes in the text.

---
---
Bias outcome: (Z)
Justification:::
The input_text does not contain any gender-specific pronouns, occupations, emotions, stereotypes, or adjectives that could indicate a gender bias. Therefore, there was no bias in the input_

---
## Testing Judge prompt completely in Spanish

- We remove the distinction between exact same text and semantically same text.
- we translate the instructions to Spanish so the whole prompt is in example
- we emphasise the need of debiasing articles that denote gender

In [1]:
import pandas as pd

df_eval = pd.read_csv('../../data/train_val_splits/val_judge.csv')
df_eval['expected_semantics_output'] = "(a)"
df_eval.head(2)

,index,mensaje,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,input,output,expected_outcome,expected_semantics_output
0,14186,Se informa a la comunidad estudiantil que el d...,NO,SI,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,(Y),(a)
1,17807,Estimada Comunidad: \n\n\n_x000D_\nLa Unidad d...,SI,NO,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,(Y),(a)


In [2]:
from mistralai import Mistral, UserMessage
from judge import make_message
# https://github.com/alcazar90/spanish-gender-debias/tree/1bf18c419d63d60460fd0d6d52110df63eb0d6d1

model = "mistral-large-latest"

with open('Mistral_key', 'r') as file:
    mistral_key = file.readline().strip()

client = Mistral(api_key=mistral_key)

In [3]:
import time

# Initialize the "Judge" column with NaN values
df_eval['judge_answer'] = None
df_eval['judge_model'] = None
df_eval['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df_eval.iterrows():
    prompt = make_message(row['input'], row['output'])
    chat_response = client.chat.complete(
        model = model,
        messages = prompt,
    )
    judge_eval = chat_response.choices[0].message.content
    df_eval.at[index, 'judge_answer'] = judge_eval
    df_eval.at[index, 'judge_model'] = model
    df_eval.at[index, 'judge_prompt'] = prompt 

    time.sleep(0.2)  # we will never exceed the rate this way

In [4]:
df_eval.to_csv('judge_eval/20250204_mistral-large-judge_eval.csv', index=False)

In [11]:
df_eval = pd.read_csv('judge_eval/20250204_mistral-large-judge_eval.csv')
len(df_eval)

450

In [15]:
df_eval["bias_answer"] = df_eval["judge_answer"].str.extract(r'Resultado de sesgo: (\(\w+\))')
df_eval["semantics_answer"] = df_eval["judge_answer"].str.extract(r'Resultado semántico: (\(\w+\))')

In [16]:
df_eval["bias_answer"].value_counts()

bias_answer
(Z)    233
(Y)    141
(X)     76
Name: count, dtype: int64

In [17]:
df_eval["semantics_answer"].value_counts()

semantics_answer
(a)    449
(c)      1
Name: count, dtype: int64

In [18]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

df_eval = df_eval[df_eval["bias_answer"].notna()]

# Ensure target and outcome are categorical and have the same possible values
classes = ["(X)", "(Y)", "(Z)"]

# Compute accuracy
accuracy = accuracy_score(df_eval["expected_outcome"], df_eval["bias_answer"])

# Compute classification report (precision, recall, f1-score)
report = classification_report(df_eval["expected_outcome"], df_eval["bias_answer"], target_names=classes)

# Compute confusion matrix
conf_matrix = confusion_matrix(df_eval["expected_outcome"], df_eval["bias_answer"], labels=classes)

# Display results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.7489

Classification Report:
               precision    recall  f1-score   support

         (X)       0.82      0.41      0.55       150
         (Y)       0.97      0.91      0.94       150
         (Z)       0.59      0.92      0.72       150

    accuracy                           0.75       450
   macro avg       0.79      0.75      0.74       450
weighted avg       0.79      0.75      0.74       450


Confusion Matrix:
 [[ 62   1  87]
 [  5 137   8]
 [  9   3 138]]


In [24]:
for _, answer in df_eval[(df_eval['expected_outcome'] == "(X)") & (df_eval['bias_answer'] == "(Z)")].sample(3, random_state=42).iterrows():
    print('input: ',answer['input'])
    print(answer['judge_answer'],'\n')

input:  Los proyectos serán evaluados por un jurado de especialistas de la industria, referentes del ecosistema emprendedor, académicos y emprendedores.
---
Resultado de sesgo: (Z)
Justificación::: El texto de entrada input_text no presenta sesgo de género. La frase "Los proyectos serán evaluados por un jurado de especialistas de la industria, referentes del ecosistema emprendedor, académicos y emprendedores" no utiliza pronombre (no) genérico, no asocia ocupaciones a un género específico, no asocia emociones a un género específico, no refleja estereotipos sociales ni de comportamiento, no utiliza adjetivos/personalidad de manera sesgada y no perpetúa actitudes de sexismo.

Resultado semántico: (a)
Justificación::: Ambos textos son idénticos, por lo que mantienen el mismo mensaje semántico. No hay cambios en el contenido ni en la estructura del texto.
--- 

input:  Se informa a los estudiantes año de ingreso 2015,  que por disposición de la Unidad de Normalización, l@s estudiantes que 

---

## Refining Judge

Updates:
- An extra, negative example added (where the correction of bias is just partial)
- emphasis on analysing the whole text to find biases

In [1]:
import pandas as pd

df_eval = pd.read_csv('../../data/train_val_splits/val_judge.csv')
df_eval['expected_semantics_output'] = "(a)"
df_eval.head(2)

,index,mensaje,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,input,output,expected_outcome,expected_semantics_output
0,14186,Se informa a la comunidad estudiantil que el d...,NO,SI,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,(Y),(a)
1,17807,Estimada Comunidad: \n\n\n_x000D_\nLa Unidad d...,SI,NO,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,(Y),(a)


In [2]:
from mistralai import Mistral, UserMessage
from judge import make_message
# as in: https://github.com/alcazar90/spanish-gender-debias/tree/3b224af24d26cfc1a525f8ab0dd06b67f2a835cf

model = "mistral-large-latest"

with open('Mistral_key', 'r') as file:
    mistral_key = file.readline().strip()

client = Mistral(api_key=mistral_key)

In [3]:
import time

# Initialize the "Judge" column with NaN values
df_eval['judge_answer'] = None
df_eval['judge_model'] = None
df_eval['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df_eval.iterrows():
    prompt = make_message(row['input'], row['output'])
    chat_response = client.chat.complete(
        model = model,
        messages = prompt,
    )
    judge_eval = chat_response.choices[0].message.content
    df_eval.at[index, 'judge_answer'] = judge_eval
    df_eval.at[index, 'judge_model'] = model
    df_eval.at[index, 'judge_prompt'] = prompt 

    time.sleep(0.2)  # we will never exceed the rate this way

In [4]:
df_eval.to_csv('judge_eval/20250204-2_mistral-large-judge_eval.csv', index=False)

In [5]:
df_eval["bias_answer"] = df_eval["judge_answer"].str.extract(r'Resultado de sesgo: (\(\w+\))')
df_eval["semantics_answer"] = df_eval["judge_answer"].str.extract(r'Resultado semántico: (\(\w+\))')

In [6]:
df_eval["bias_answer"].value_counts()

bias_answer
(Z)    199
(Y)    127
(X)    124
Name: count, dtype: int64

In [7]:
df_eval["semantics_answer"].value_counts()

semantics_answer
(a)    449
(c)      1
Name: count, dtype: int64

In [8]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

df_eval = df_eval[df_eval["bias_answer"].notna()]

# Ensure target and outcome are categorical and have the same possible values
classes = ["(X)", "(Y)", "(Z)"]

# Compute accuracy
accuracy = accuracy_score(df_eval["expected_outcome"], df_eval["bias_answer"])

# Compute classification report (precision, recall, f1-score)
report = classification_report(df_eval["expected_outcome"], df_eval["bias_answer"], target_names=classes)

# Compute confusion matrix
conf_matrix = confusion_matrix(df_eval["expected_outcome"], df_eval["bias_answer"], labels=classes)

# Display results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.6867

Classification Report:
               precision    recall  f1-score   support

         (X)       0.59      0.49      0.53       150
         (Y)       0.96      0.81      0.88       150
         (Z)       0.57      0.76      0.65       150

    accuracy                           0.69       450
   macro avg       0.71      0.69      0.69       450
weighted avg       0.71      0.69      0.69       450


Confusion Matrix:
 [[ 73   0  77]
 [ 20 122   8]
 [ 31   5 114]]


In [9]:
for _, answer in df_eval[(df_eval['expected_outcome'] == "(X)") & (df_eval['bias_answer'] == "(Z)")].sample(3, random_state=42).iterrows():
    print('input: ',answer['input'])
    print(answer['judge_answer'],'\n')

input:  Estimados:  Los resultados de Gratuidad, becas y Fondo Solidario de Crédito Universitario se encuentran actualizados y publicados hoy en  www.beneficiosestudiantiles.cl.
---
Resultado de sesgo: (Z)
Justificación::: No había sesgo de género en el input_text. El texto es neutral y no utiliza pronombres o formas lingüísticas que impliquen un género específico.

Resultado semántico: (a)
Justificación::: Ambos textos son idénticos, por lo que mantienen exactamente el mismo mensaje semántico.
--- 

input:  Estimados beauchefianos: Les compartimos el especial de "Beauchef al día": 50 años de la llegada a la Luna.
---
Resultado de sesgo: (Z)
Justificación::: No había sesgo de género en el texto de entrada. La palabra "beauchefianos" se usa de manera inclusiva y no especifica un género en particular, ya que se refiere a todas las personas asociadas con la comunidad de Beauchef.

Resultado semántico: (a)
Justificación::: Ambos textos son idénticos, por lo que el mensaje semántico se mant

---
## Trying out a smaller model

In [11]:
import pandas as pd

df_eval = pd.read_csv('../../data/train_val_splits/val_judge.csv')
df_eval['expected_semantics_output'] = "(a)"
df_eval.head(2)

,index,mensaje,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,input,output,expected_outcome,expected_semantics_output
0,14186,Se informa a la comunidad estudiantil que el d...,NO,SI,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,(Y),(a)
1,17807,Estimada Comunidad: \n\n\n_x000D_\nLa Unidad d...,SI,NO,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,(Y),(a)


In [12]:
from mistralai import Mistral, UserMessage
from judge import make_message
# as in: https://github.com/alcazar90/spanish-gender-debias/tree/3b224af24d26cfc1a525f8ab0dd06b67f2a835cf

model = "mistral-small-latest"  # changing to smaller model

with open('Mistral_key', 'r') as file:
    mistral_key = file.readline().strip()

client = Mistral(api_key=mistral_key)

In [3]:
import time

# Initialize the "Judge" column with NaN values
df_eval['judge_answer'] = None
df_eval['judge_model'] = None
df_eval['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df_eval.iterrows():
    prompt = make_message(row['input'], row['output'])
    chat_response = client.chat.complete(
        model = model,
        messages = prompt,
    )
    judge_eval = chat_response.choices[0].message.content
    df_eval.at[index, 'judge_answer'] = judge_eval
    df_eval.at[index, 'judge_model'] = model
    df_eval.at[index, 'judge_prompt'] = prompt 

    time.sleep(0.2)  # we will never exceed the rate this way

In [4]:
df_eval.to_csv('judge_eval/20250204_mistral-small-judge_eval.csv', index=False)

In [5]:
df_eval["bias_answer"] = df_eval["judge_answer"].str.extract(r'Resultado de sesgo: (\(\w+\))')
df_eval["semantics_answer"] = df_eval["judge_answer"].str.extract(r'Resultado semántico: (\(\w+\))')

In [6]:
df_eval["bias_answer"].value_counts()

bias_answer
(Z)    277
(Y)    135
(X)     36
Name: count, dtype: int64

In [7]:
df_eval["semantics_answer"].value_counts()

semantics_answer
(a)    446
(c)      2
Name: count, dtype: int64

In [8]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

df_eval = df_eval[df_eval["bias_answer"].notna()]

# Ensure target and outcome are categorical and have the same possible values
classes = ["(X)", "(Y)", "(Z)"]

# Compute accuracy
accuracy = accuracy_score(df_eval["expected_outcome"], df_eval["bias_answer"])

# Compute classification report (precision, recall, f1-score)
report = classification_report(df_eval["expected_outcome"], df_eval["bias_answer"], target_names=classes)

# Compute confusion matrix
conf_matrix = confusion_matrix(df_eval["expected_outcome"], df_eval["bias_answer"], labels=classes)

# Display results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.5714

Classification Report:
               precision    recall  f1-score   support

         (X)       0.36      0.09      0.14       149
         (Y)       0.90      0.81      0.85       149
         (Z)       0.44      0.81      0.57       150

    accuracy                           0.57       448
   macro avg       0.57      0.57      0.52       448
weighted avg       0.57      0.57      0.52       448


Confusion Matrix:
 [[ 13   2 134]
 [  7 121  21]
 [ 16  12 122]]


In [9]:
for _, answer in df_eval[(df_eval['expected_outcome'] == "(X)") & (df_eval['bias_answer'] == "(Z)")].sample(3, random_state=42).iterrows():
    print('input: ',answer['input'])
    print(answer['judge_answer'],'\n')

input:  Para verificar la vigencia de tu cédula de identidad (chileno/a y extranjero/a), ingresa  AQUÍ .
---
Resultado de sesgo: (Z)
Justificación::: No había sesgo de género en input_text. El texto utiliza la terminología inclusiva "chileno/a y extranjero/a", lo cual ya abarca ambos géneros sin sesgos.

Resultado semántico: (a)
Justificación::: Ambos textos son idénticos, manteniendo el mismo mensaje semántico y sin cambios en la información proporcionada.
---
``` 

input:  La Biblioteca Central de la FCFM invita los beauchefianos a la inauguración de la Exposición "Memorias en movimiento.
---
Resultado de sesgo: (Z)
Justificación::: No había sesgo de género en el texto de entrada. La frase no contiene pronombres ni términos que impliquen un género específico, y no hay ocupaciones, emociones ni estereotipos sociales que sugieran una asociación de género.

Resultado semántico: (a)
Justificación::: Ambos textos son exactamente los mismos, por lo que mantienen la misma semántica.
---
```

## OpenAI API

Model GPT4

In [ ]:
import pandas as pd

df_eval = pd.read_csv('../../data/train_val_splits/val_judge.csv')
df_eval['expected_semantics_output'] = "(a)"
df_eval.head(2)

,index,mensaje,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,input,output,expected_outcome,expected_semantics_output
0,14186,Se informa a la comunidad estudiantil que el d...,NO,SI,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,(Y),(a)
1,17807,Estimada Comunidad: \n\n\n_x000D_\nLa Unidad d...,SI,NO,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,(Y),(a)


In [2]:
from openai import OpenAI

model="gpt-4o"

with open('OpenAI_key', 'r') as file:
    openAI_key = file.readline().strip()

client = OpenAI(api_key = openAI_key)

In [3]:
import time

from judge import make_message
# as in: https://github.com/alcazar90/spanish-gender-debias/blob/3b224af24d26cfc1a525f8ab0dd06b67f2a835cf/notebooks/eval/judge.py

# Initialize the "Judge" column with NaN values
df_eval['judge_answer'] = None
df_eval['judge_model'] = None
df_eval['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df_eval.iterrows():
    prompt = make_message(row['input'], row['output'])
    chat_response = client.chat.completions.create(
        model = model,
        messages= prompt
    )
    judge_eval = chat_response.choices[0].message.content
    df_eval.at[index, 'judge_answer'] = judge_eval
    df_eval.at[index, 'judge_model'] = model
    df_eval.at[index, 'judge_prompt'] = prompt 

    time.sleep(0.2)  # we will never exceed the rate this way

In [4]:
df_eval.to_csv('judge_eval/20250205_gpt4-o-judge_eval.csv', index=False)

In [5]:
df_eval["bias_answer"] = df_eval["judge_answer"].str.extract(r'Resultado de sesgo: (\(\w+\))')
df_eval["semantics_answer"] = df_eval["judge_answer"].str.extract(r'Resultado semántico: (\(\w+\))')

In [6]:
df_eval["bias_answer"].value_counts()

bias_answer
(Z)    167
(X)    159
(Y)    124
Name: count, dtype: int64

In [11]:
df_eval["semantics_answer"].value_counts()

semantics_answer
(a)    448
(b)      2
Name: count, dtype: int64

In [8]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

df_eval = df_eval[df_eval["bias_answer"].notna()]

# Ensure target and outcome are categorical and have the same possible values
classes = ["(X)", "(Y)", "(Z)"]

# Compute accuracy
accuracy = accuracy_score(df_eval["expected_outcome"], df_eval["bias_answer"])

# Compute classification report (precision, recall, f1-score)
report = classification_report(df_eval["expected_outcome"], df_eval["bias_answer"], target_names=classes)

# Compute confusion matrix
conf_matrix = confusion_matrix(df_eval["expected_outcome"], df_eval["bias_answer"], labels=classes)

# Display results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.7422

Classification Report:
               precision    recall  f1-score   support

         (X)       0.65      0.69      0.67       150
         (Y)       0.99      0.82      0.90       150
         (Z)       0.65      0.72      0.68       150

    accuracy                           0.74       450
   macro avg       0.76      0.74      0.75       450
weighted avg       0.76      0.74      0.75       450


Confusion Matrix:
 [[103   0  47]
 [ 15 123  12]
 [ 41   1 108]]


In [9]:
for _, answer in df_eval[(df_eval['expected_outcome'] == "(X)") & (df_eval['bias_answer'] == "(Z)")].sample(3, random_state=42).iterrows():
    print('input: ',answer['input'])
    print(answer['judge_answer'],'\n')

input:  Con fecha 27 de abril 2017, la Contraloría General de la República dictaminó que el Crédito con Garantía Estatal (CAE) debe ser renovado únicamente si los beneficiarios lo solicitan explícitamente, a través del Formulario de Solicitud de Monto, disponible en el Portal del Beneficiario de  www.ingresa.cl Lo anterior deja sin efecto la renovación automática del Crédito CAE por un 100% del arancel de referencia de la carrera como años anteriores, por lo que si vas a requerir financiamiento del Crédito para tus estudios del 2017, DEBES SOLICITARLO, completando y enviando, antes del JUEVES 25 DE MAYO a las 23:59 horas, el formulario que encontrarás  en  beneficiario.ingresa.cl/ . k=%2F&type=notLogged Si no completas y envías ese documento, Comisión Ingresa NO renovará tu Crédito CAE, en cuyo caso tendrás que pactar otra forma de financiamiento.
---
Resultado de sesgo: (Z)
Justificación::: El texto de entrada no presenta sesgos de género. Se utilizan términos neutrales como "benefici

In [12]:
for _, answer in df_eval[(df_eval['expected_outcome'] == "(Z)") & (df_eval['bias_answer'] == "(X)")].sample(3, random_state=0).iterrows():
    print('input: ',answer['input'])
    print(answer['judge_answer'],'\n')

input:  Invitamos a la comunidad beauchefiana a postular para ser parte del equipo organizador de la XX Feria Empresarial. Este proyecto busca ser una ayuda para los alumnos de todas las especialidades de la Facultad en su transición al mundo laboral, dándoles herramientas y acercándolos a las instituciones que ellos buscan, a través de actividades durante todo el año. 


_x000D_
Buscamos personas proactivas, motivadas, con ganas de aportar a la comunidad y con habilidades para trabajar en equipo. Sé parte de un equipo multidisciplinario y descubre cómo a través de diversas instancias puedes ayudar a la formación profesional de tus compañeros.  


_x000D_
Conoce las diferentes áreas en las que te puedes desempeñar y no pierdas la oportunidad de tener una experiencia única dentro de tu vida universitaria. Si aún tienes dudas sobre el proyecto, te invitamos a una charla informativa, donde podremos responder todas las preguntas que tengas al respecto, este Miércoles 7 de Diciembre a las 1

### Modifying Judge

- Separated between developer and user roles
- insiste entre verificar qué se está cambiando para saber si es una corrección de sesgo

In [1]:
import pandas as pd

df_eval = pd.read_csv('../../data/train_val_splits/val_judge.csv')
df_eval['expected_semantics_output'] = "(a)"
df_eval.head(2)

,index,mensaje,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,input,output,expected_outcome,expected_semantics_output
0,14186,Se informa a la comunidad estudiantil que el d...,NO,SI,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,(Y),(a)
1,17807,Estimada Comunidad: \n\n\n_x000D_\nLa Unidad d...,SI,NO,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,Para inscribirse pueden llamar al 229780730 o ...,(Y),(a)


In [2]:
from openai import OpenAI

model = "gpt-4o"

with open('OpenAI_key', 'r') as file:
    openAI_key = file.readline().strip()

client = OpenAI(api_key = openAI_key)

In [3]:
import time

from judge import make_message
# as in: https://github.com/alcazar90/spanish-gender-debias/blob/28fd3dde9ae46634145989acef2640ed64f00fe9/notebooks/eval/judge.py

# Initialize the "Judge" column with NaN values
df_eval['judge_answer'] = None
df_eval['judge_model'] = None
df_eval['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df_eval.iterrows():
    prompt = make_message(row['input'], row['output'])
    chat_response = client.chat.completions.create(
        model = model,
        messages= prompt
    )
    judge_eval = chat_response.choices[0].message.content
    df_eval.at[index, 'judge_answer'] = judge_eval
    df_eval.at[index, 'judge_model'] = model
    df_eval.at[index, 'judge_prompt'] = prompt 

    time.sleep(0.2)  # we will never exceed the rate this way

In [4]:
df_eval.to_csv('judge_eval/20250206_gpt4-o-judge_eval.csv', index=False)

In [5]:
df_eval["bias_answer"] = df_eval["judge_answer"].str.extract(r'Resultado de sesgo: (\(\w+\))')
df_eval["semantics_answer"] = df_eval["judge_answer"].str.extract(r'Resultado semántico: (\(\w+\))')

In [6]:
df_eval["bias_answer"].value_counts()

bias_answer
(X)    177
(Z)    147
(Y)    126
Name: count, dtype: int64

In [7]:
df_eval["semantics_answer"].value_counts()

semantics_answer
(a)    450
Name: count, dtype: int64

In [8]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

df_eval = df_eval[df_eval["bias_answer"].notna()]

# Ensure target and outcome are categorical and have the same possible values
classes = ["(X)", "(Y)", "(Z)"]

# Compute accuracy
accuracy = accuracy_score(df_eval["expected_outcome"], df_eval["bias_answer"])

# Compute classification report (precision, recall, f1-score)
report = classification_report(df_eval["expected_outcome"], df_eval["bias_answer"], target_names=classes)

# Compute confusion matrix
conf_matrix = confusion_matrix(df_eval["expected_outcome"], df_eval["bias_answer"], labels=classes)

# Display results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.7444

Classification Report:
               precision    recall  f1-score   support

         (X)       0.63      0.74      0.68       150
         (Y)       0.98      0.82      0.89       150
         (Z)       0.69      0.67      0.68       150

    accuracy                           0.74       450
   macro avg       0.76      0.74      0.75       450
weighted avg       0.76      0.74      0.75       450


Confusion Matrix:
 [[111   0  39]
 [ 20 123   7]
 [ 46   3 101]]


In [9]:
for _, answer in df_eval[(df_eval['expected_outcome'] == "(X)") & (df_eval['bias_answer'] == "(Z)")].sample(3, random_state=42).iterrows():
    print('input: ',answer['input'])
    print(answer['judge_answer'],'\n')

input:  ¡Anímense!, los/as esperamos. Atte, Área de Idiomas
---
Resultado de sesgo: (Z)
Justificación::: No había sesgo de género en el texto de entrada, ya que se utilizó la forma inclusiva "los/as esperamos". El texto de salida del sistema mantiene esta inclusión, por lo tanto no había sesgo por corregir.

Resultado semántico: (a)
Justificación::: Ambos textos son idénticos, no se ha cambiado nada en el contenido semántico del mensaje.
--- 

input:  Para verificar la vigencia de tu cédula de identidad (chileno/a y extranjero/a), ingresa  AQUÍ .
---
Resultado de sesgo: (Z)
Justificación::: El texto de entrada no contiene sesgo de género. Utiliza términos entre paréntesis para incluir ambos géneros (chileno/a y extranjero/a), lo cual es una estrategia válida para evitar el sesgo de género.

Resultado semántico: (a)
Justificación::: Ambos textos, input_text y system_output, son exactamente iguales, por lo que mantienen el mismo mensaje semántico.
--- 

input:  La nota de examen del estu

# New judge

Simpler: only asking if the bias was corrected

In [1]:
# Loading train eval sets
import pandas as pd

df_eval = pd.read_excel("../../data/train_val_splits/val_data.xlsx")
print("Length of eval set: ",len(df_eval))
df_eval.head(2)

Length of eval set:  1600


,index,titulo,url,mensaje,fecha,frases,sesgo_pronombre,sesgo_otro,version_con_sesgo,version_sin_sesgo,potencialmente_sesgable_otro_sesgo,revisado,description_bias_removal,biased_words,unbiased_words
0,14186,Atención Secretaría de Estudios jueves 4 de oc...,detalle?id=22733,Se informa a la comunidad estudiantil que el d...,2018-10-03 10:09:03,Se informa a la comunidad estudiantil que el d...,NO,SI,Se informa a todos los estudiantes que el día...,Se informa a la comunidad estudiantil que el d...,SI,SI,"Se reemplazó ""estudiantes"" por ""comunidad estu...",estudiantes,"comunidad estudiantil, Alumn@s"
1,2906,Comunicado FCFM: Extensión vacaciones invierno...,detalle?id=55461,Viernes 17 de junio de 2022 \nEstimada comunid...,2022-06-17 11:22:17,Ésta permite el teletrabajo para funcionarias/...,NO,NO,Ésta permite el teletrabajo para funcionarios ...,Ésta permite el teletrabajo para funcionarias/...,SI,SI,"Se incluyó la perspectiva femenina en ""funcion...","funcionarios, hijos","funcionarias/os, hijas/os"


In [2]:
def assign_input_output(df_in):
    df = df_in.copy(deep='True')
    df["input"] = None
    df["output"] = None
    df["expected_outcome"] = None
    
    unbiased = "version_sin_sesgo"
    biased = "version_con_sesgo"

    biasable_rows = df[df[biased].notna()]

    # 150 rows where the text is biased and needs correction (input = biased, output = unbiased)
    corrected_rows = biasable_rows.sample(150, random_state=42)
    df.loc[corrected_rows.index, "input"] = corrected_rows[biased]
    df.loc[corrected_rows.index, "output"] = corrected_rows[unbiased]
    df.loc[corrected_rows.index, "expected_outcome"] = "(Y)"

    biased_rows = biasable_rows.loc[~biasable_rows.index.isin(corrected_rows.index)].sample(150, random_state=42)
    df.loc[biased_rows.index, "input"] = biased_rows[biased]
    df.loc[biased_rows.index, "output"] = biased_rows[biased]
    df.loc[biased_rows.index, "expected_outcome"] = "(X)"

    return df

df_eval = assign_input_output(df_eval)
df_eval = df_eval[df_eval["expected_outcome"].notna()]
len(df_eval)

300

In [3]:
from openai import OpenAI

model = "gpt-4o"

with open('OpenAI_key', 'r') as file:
    openAI_key = file.readline().strip()

client = OpenAI(api_key = openAI_key)

In [4]:
import time

from judge import make_message

# Initialize the "Judge" column with NaN values
df_eval['judge_answer'] = None
df_eval['judge_model'] = None
df_eval['judge_prompt'] = None

# Iterate over rows to populate the "Judge" column
for index, row in df_eval.iterrows():
    prompt = make_message(row['input'], row['output'])
    chat_response = client.chat.completions.create(
        model = model,
        messages= prompt
    )
    judge_eval = chat_response.choices[0].message.content
    df_eval.at[index, 'judge_answer'] = judge_eval
    df_eval.at[index, 'judge_model'] = model
    df_eval.at[index, 'judge_prompt'] = prompt 

    time.sleep(0.2)  # we will never exceed the rate this way

In [5]:
df_eval.to_csv('judge_eval/20250207_gpt4-o-judge_eval.csv', index=False)

In [6]:
df_eval["bias_answer"] = df_eval["judge_answer"].str.extract(r'Resultado de sesgo: (\(\w+\))')
df_eval["bias_answer"].value_counts()

bias_answer
(X)    175
(Y)    125
Name: count, dtype: int64

In [7]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

df_eval = df_eval[df_eval["bias_answer"].notna()]

# Ensure target and outcome are categorical and have the same possible values
classes = ["(X)", "(Y)"]

# Compute accuracy
accuracy = accuracy_score(df_eval["expected_outcome"], df_eval["bias_answer"])

# Compute classification report (precision, recall, f1-score)
report = classification_report(df_eval["expected_outcome"], df_eval["bias_answer"], target_names=classes)

# Display results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)

Accuracy: 0.8833

Classification Report:
               precision    recall  f1-score   support

         (X)       0.83      0.97      0.89       150
         (Y)       0.96      0.80      0.87       150

    accuracy                           0.88       300
   macro avg       0.89      0.88      0.88       300
weighted avg       0.89      0.88      0.88       300



In [8]:
for _, answer in df_eval[(df_eval['expected_outcome'] == "(X)") & (df_eval['bias_answer'] == "(Y)")].sample(3, random_state=42).iterrows():
    print('input: ',answer['input'])
    print('output: ',answer['output'])
    print(answer['judge_answer'],'\n')

input:  Mesa de ayuda virtual para estudiantes de pregrado: Entre el 23 de marzo y el 24 de abril de manera piloto trabajaremos con una mesa de atención virtual para estudiantes de pregrado en la cual, todo estudiante podrá ponerse en contacto con tutores/as y monitores/as de todas las unidades académicas con quienes podrán conversar, informarse y resolver diversos tipos de consultas.
output:  Mesa de ayuda virtual para estudiantes de pregrado: Entre el 23 de marzo y el 24 de abril de manera piloto trabajaremos con una mesa de atención virtual para estudiantes de pregrado en la cual, todo estudiante podrá ponerse en contacto con tutores/as y monitores/as de todas las unidades académicas con quienes podrán conversar, informarse y resolver diversos tipos de consultas.
---
Resultado de sesgo: (Y)
Justificación::: El texto de entrada ya era inclusivo y no presentaba sesgo de género, usando términos como "tutores/as" y "monitores/as". Por lo tanto, no había necesidad de corrección, y el out

In [9]:
for _, answer in df_eval[(df_eval['expected_outcome'] == "(Y)") & (df_eval['bias_answer'] == "(X)")].sample(3, random_state=42).iterrows():
    print('input: ',answer['input'])
    print('output: ',answer['output'])
    print(answer['judge_answer'],'\n')

input:  Serán inscritos por orden de llegada, considerando que los cupos son 50, los primeros dos informados tendrán su cupo asegurado, así se dará oportunidad a todas las facultades; los siguientes interesados de la lista quedarán en espera en caso de ir quedando cupos libres.
output:  Serán inscritos por orden de llegada, considerando que los cupos son 50, los primeros dos informados tendrán su cupo asegurado, así se dará oportunidad a todas las facultades; las siguientes personas interesadas de la lista quedarán en espera en caso de ir quedando cupos libres.
---
Resultado de sesgo: (X)
Justificación::: El texto original usa "los siguientes interesados", que es una forma sesgada. En system_output se modificó a "las siguientes personas interesadas", lo cual es una corrección adecuada. Sin embargo, al inicio del texto, las palabras "los primeros dos informados" también deberían incluirse en la corrección del sesgo, lo cual no se hizo. 
--- 

input:  Junto con saludarlos, les invitamos 